In [1]:
import numpy as np
import random
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [ ]:
# Define SARSA parameters
alpha = 0.1  # Learning rate
gamma = 0.95  # Discount factor
epsilon = 0.1  # Exploration rate (we will decay this over time for more exploitation)

# Define BEO parameters
population_size = 50  # Number of individuals in the population
n_iterations = 100  # Number of optimization iterations
penalty_weight = 0.15  # Penalty weight for selecting too many features

In [ ]:
# Load the dataset
file_path = "/kaggle/input/pcos-ml/PCOS_data_without_infertility.xlsx"
df = pd.read_excel(file_path, sheet_name="Full_new")

# Data Preprocessing
df = df.drop(columns=['Sl. No', 'Patient File No.', 'Unnamed: 44'])
df = df.apply(pd.to_numeric, errors='coerce')
df.fillna(df.median(), inplace=True)

In [ ]:
# Encode categorical columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
categorical_columns = ['Blood Group', 'Cycle(R/I)', 'Pregnant(Y/N)', 
                       'Weight gain(Y/N)', 'hair growth(Y/N)', 
                       'Skin darkening (Y/N)', 'Hair loss(Y/N)', 
                       'Pimples(Y/N)', 'Fast food (Y/N)', 
                       'Reg.Exercise(Y/N)']
for col in categorical_columns:
    if col in df.columns:
        df[col] = le.fit_transform(df[col])

# Split into features and target
X = df.drop(columns=['PCOS (Y/N)'])
y = df['PCOS (Y/N)']

# Assuming these are the selected features from ensemble
selected_features = ['Follicle No. (L)', 'hair growth(Y/N)', 'Follicle No. (R)', 'Cycle(R/I)', 'Fast food (Y/N)', 
                      'AMH(ng/mL)', 'Skin darkening (Y/N)', 'Weight gain(Y/N)', 'Pimples(Y/N)', 'Cycle length(days)', 
                      'Hip(inch)', 'Weight (Kg)', 'FSH/LH', 'FSH(mIU/mL)']
n_features = len(selected_features)
n_actions = n_features  # One action per feature (either include or exclude)

# Initialize population (binary representation of feature subsets)
population = np.random.randint(2, size=(population_size, n_features))

# SARSA Q-table initialization
Q = np.zeros((population_size, n_actions))


In [ ]:
def evaluate_solution(solution, X_train, y_train, X_test, y_test):
    """Evaluate the accuracy of a solution (feature subset) and penalize for too many features."""
    # Use only the selected features from the solution
    selected_columns = [selected_features[i] for i in range(len(solution)) if solution[i] == 1]
    if len(selected_columns) == 0:
        return 0  # If no features selected, return zero accuracy
    
    # Fit a simple model (e.g., XGBoost) on the selected features
    model = xgb.XGBClassifier(eval_metric='mlogloss')
    model.fit(X_train[selected_columns], y_train)
    accuracy = model.score(X_test[selected_columns], y_test)
    
    # Apply penalty for number of selected features (objective: fewer features, high accuracy)
    penalty = penalty_weight * len(selected_columns)
    
    return accuracy - penalty  # Objective: maximize accuracy, minimize features


In [ ]:
# BEO + SARSA optimization loop
for iteration in range(n_iterations):
    for i in range(population_size):
        # Select an action (flip a feature on/off)
        if random.uniform(0, 1) < epsilon:  # Exploration
            action = np.random.randint(n_actions)
        else:  # Exploitation (SARSA policy)
            action = np.argmax(Q[i])
        
        # Apply the action (flip the feature)
        new_population = np.copy(population)
        new_population[i][action] = 1 - new_population[i][action]
        
        # Evaluate the new solution
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        reward = evaluate_solution(new_population[i], X_train, y_train, X_test, y_test)
        
        # SARSA: Update Q-value
        next_action = np.argmax(Q[i])  # Choose next action (greedy)
        next_reward = evaluate_solution(new_population[i], X_train, y_train, X_test, y_test)
        Q[i][action] += alpha * (reward + gamma * next_reward - Q[i][action])
        
        # Print the current selected features for each iteration
        selected_columns = [selected_features[idx] for idx, val in enumerate(new_population[i]) if val == 1]
        print(f"Iteration {iteration+1}, Individual {i+1}, Selected Features: {selected_columns}")


In [ ]:

# Get the final solution (best individual from the population)
best_individual_idx = np.argmax([evaluate_solution(population[i], X_train, y_train, X_test, y_test) for i in range(population_size)])
final_solution = population[best_individual_idx]

# Final selected features after optimization
final_selected_features = [selected_features[idx] for idx, val in enumerate(final_solution) if val == 1]
print(f"Final Selected Features: {final_selected_features}")
